In [ ]:
import sys
import os

import pandas as pd
import mappy as mp
import pickle
from parse_gene_anno import GeneAnno
from interlap import reduce
from design_probe_m import * 
from filter_probe import filter_probes
import os
from collections import Counter
import pickle
from math import log
import mappy as mp
from Bio.SeqUtils import MeltingTemp as mt
from parse_gene_anno import GeneAnno
from interlap import reduce
import sys
import pandas as pd

# Parameter
# probe length (52bp) - is it fixed?
# acceptable GC content (45-65%), 
# minimum melting temperature (Tm) (50°C), 
# and other quality control thresholds.

# Probe Request: It reads a CSV file (Probes_request.csv) containing the list of genes (by gene symbol) for which probes are needed. It automatically finds the corresponding Ensembl gene IDs.


# parameters
data_folder = "probes/saturate_design_genes/" # 24_02_19_GENES
max_exon_probe = 1000000 # the maximum number of probes per exon
min_exon_probe = 10 # the minimum number of probes per exon, based on Kai's requirement


probe_len = 52
min_CDS_probes = 1000000000 #100
GC_range = [0.45,0.65] #[0.45,0.65]
min_Tm = 50 #50
max_Tm_diff = 8 #10
num_rep = 5 #5
min_ent = 0.9 #0.9

best_n = 24 #4
min_chain_score = 8 #16

save_top = 10


min_dist = 50 #50 #40
prime_5end = int(probe_len/2)
const_3 = 'TAGAAGAGTCTTCCTTTACG'
const_5_1 = '/5Phos/GGAGGGCAGCAAACGGAA'
const_5_2 = 'NNNNNNNNNNCGAGACTACTGCGAGTAC'


# read reference file
# human genome sequence,
# gene annotation file
# prebuilt index of the genome


data = pd.read_excel("reference/GTEx_Analysis__exon_reads_rank_normalied.xlsx")
data = data[['start','end','gene_id','gene_name','exon_rank','exon_id']]
data['gene_id'] = data['gene_id'].str.split('.').str[0]
fa_dict = pickle.load(open("reference/fa_dict_human.pkl","rb"))
genome_paths = get_reference_paths()
gtf_f = genome_paths['gtf_f']
index_f = genome_paths['index_f']

# load data
probe_request = pd.read_csv(data_folder+'Probes_request.csv')
probe_request.sort_values(by='gene_id', ascending=False, na_position='last', inplace=True)
probe_request.drop_duplicates(subset='gene symbol', keep='first', inplace=True)
probe_request.reset_index(drop=True, inplace=True)
probe_request = probe_request.applymap(lambda x: x.strip() if isinstance(x, str) else x)
missing_genes = update_probe_request_gene_ids(probe_request, data, data_folder)
for gene_symbol in missing_genes:
    print(gene_symbol)

# gene_symbol_map = {
#     "IL17": "IL17A",
#     "IL12": "IL12B",
#     "IL23": "IL23A"
# }
# for old_symbol, new_symbol in gene_symbol_map.items():
#     probe_request.loc[probe_request['gene symbol'] == old_symbol, 'gene symbol'] = new_symbol
# missing_genes = update_probe_request_gene_ids(probe_request, data, data_folder)
# for gene_symbol in missing_genes:
#     print(gene_symbol)
# probe_request = pd.read_csv(data_folder+'Updated_probe_request.csv')

In [ ]:
aln = mp.Aligner(fn_idx_in=index_f, preset="sr", best_n=best_n, min_chain_score=min_chain_score)
an = GeneAnno(gtf_f)

# Generating candidate probes
# then filter the probes with the qc function _get_probe_from_intervals
status_cnt, pickle_probe = get_exon_probe_list(an, aln, fa_dict, probe_request['gene_id'].tolist(), probe_len, min_CDS_probes, GC_range, min_Tm, max_Tm_diff, num_rep, min_ent) 
pickle.dump(pickle_probe, open(data_folder+"pickle_probe.pkl","wb"))
probe_dict = pickle.load(open(data_folder+"pickle_probe.pkl","rb"))


# select the best probes. 
# based on:
# Spacing: It ensures that selected probes for a single gene are not too close to each other, enforcing a min_dist between them.

# Prioritization: It may prioritize probes that fall within the protein-coding sequence (CDS) of an exon.

# Best of the Best: If there are still too many good probes, it sorts them by how close their GC content is to an ideal value (e.g., 55%) and picks the top candidates.

# Final Count: It ensures each gene ends up with a specific number of probes (e.g., between min_exon_probe and max_exon_probe).


selected_probe = filter_probes(probe_dict, max_exon_probe, min_exon_probe, min_dist)
selected_probe_exon = get_matching_exons(selected_probe, data)


# Finally, the save_design_probes function prepares the output files.

# Probe Assembly: For each selected 52 bp homology sequence, it attaches constant adapter sequences (const_3 and const_5_1/const_5_2) to create the final, full-length probe sequences that will be synthesized. This step creates the two separate oligos (probe_3 and probe_5) needed for the assay.

# Saving Results: It writes the results to CSV files:

# Probe_exon_design_all.csv: Contains all the successfully designed probes.

# Probe_exon_design_top.csv: A smaller, curated list containing the top save_top (e.g., 10) probes for each gene, ready for ordering.

save_design_probes(data_folder, probe_request, selected_probe_exon, const_3, const_5_1, const_5_2, save_top=save_top, filter_same_exon=False)